# Ebay Car Listings - Data Cleaning Project

The purpose of this project is to practice recently learned techniques with the vectorized operations that exist in the Pandas and NumPy libraries that make working and manipulating data quicker and more efficient.

The first thing I will do is import the libraries mentioned above, than use the .head() and .info() method to give me some insight on how the data is layed out and if there are any discrepencies in the quantity of data.

In [1]:
import numpy as np
import pandas as pd

autos = pd.read_csv("autos.csv", encoding = "Latin-1")

In [2]:
autos.head()
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

### Data Information

Analysing the .info() method results, it appears as though several pieces of data are missing. Most columns have 50,000 data entries, although a few of the columns (i.e. vehicleType, gearbox, model, fuelType, and notRepairedDamage) seem to have significantly less data on some instances. I will need to further investigate those columns to determine an appropriate method for correcting the miss entries.


In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
corrected_columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'regestration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'regestration_month', 'fuel_type', 'brand',
       'unrepaired_damaage', 'ad_created', 'number_of_pictures', 'postal_code',
       'last_seen']
autos.columns = corrected_columns
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,regestration_year,gearbox,power_ps,model,odometer,regestration_month,fuel_type,brand,unrepaired_damaage,ad_created,number_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Converting from camelcase to snakecase

In the cells above, I utilized the .columns attribute of the autos dataframe to view an array of the column names. I then manually corrected each to be of type snakecase. This is mostly done make the column names easier to read and more quickly recognisable by others.

In [5]:
#Create 2 functions that will clean the odometer and price data.
def cleanOdometer(od_reading):
    od_reading = od_reading.replace("km","")
    od_reading = od_reading.replace(",","")
    
    return od_reading

def cleanPrice(price):
    price = price.replace(",","")
    price = price.replace("$","")
    
    return price

#Define empty list to store odometer and price data after being cleaned of extra symbols
cleaned_od_readings = []
cleaned_prices = []

old_od = autos.loc[:,"odometer"]
old_prices = autos.loc[:,"price"]

#loop through odometer and prices. Cleaning commas, dollar signs and km values
for x in old_od:
    new_od = cleanOdometer(x)
    cleaned_od_readings.append(new_od)
    
for y in old_prices:
    new_price = cleanPrice(y)
    cleaned_prices.append(new_price)

#replace existing odometer and price series of autos dataframe with newly cleaned array
autos.loc[:,"odometer"] = cleaned_od_readings
autos.loc[:,"price"] = cleaned_prices

#convert odometer and prices columns in autos df to type int
autos.loc[:,"odometer"] = autos.loc[:,"odometer"].astype(dtype ='float')
autos.loc[:,"price"] = autos.loc[:,"price"].astype('float')

autos.rename({"odometer":"odometer_km"}, axis = 1, inplace = True)

autos.describe(include = 'all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,regestration_year,gearbox,power_ps,model,odometer_km,regestration_month,fuel_type,brand,unrepaired_damaage,ad_created,number_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,5.000000e+04,50000,44905,50000.000000,47320,50000.000000,47242,50000.000000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 18:39:34,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,NaN,25756,12859,NaN,36993,NaN,4024,NaN,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,9.840044e+03,NaN,NaN,2005.073280,NaN,116.355920,NaN,125732.700000,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,4.811044e+05,NaN,NaN,105.712813,NaN,209.216627,NaN,40042.211706,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1.100000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


## Data Cleaning - Analysing What We've Got

We start the data cleaning process by using the dataframe.describe() attribute to potentially spot any issues or irrelevancies with the numerical data in the dataframe. We note that the majority of the statistics for the number_of_pictures column are empty - a potential candidate for a column to be dropped. Other things to note:

- the min and max regestration years are 100 and 9999.
- min and max vehicle powers are 0 and 17700 PS (equivalent to bhp) respectively.
- all values for seller are the same and equal to "private"
- all values for offer_type are the same and equal to "Angebot"

In [6]:
unique_od = autos.loc[:,'odometer_km'].unique().shape
od_counts = autos.loc[:,'odometer_km'].value_counts().sort_index(ascending = False)
od_desc = autos.loc[:,'odometer_km'].describe()

print ("ODOMETER READING INFORMATION")
print("size:", unique_od)
print(od_counts)
print(od_desc)

unique_price = autos.loc[:,'price'].unique().shape
price_counts = autos.loc[:,'price'].value_counts().sort_index(ascending = False).head(10)
price_desc = autos.loc[:,'price'].describe()

print('\n')
print ("PRICE INFORMATION")
print(unique_price)
print(price_counts)
print(price_desc)

ODOMETER READING INFORMATION
size: (13,)
150000.0    32424
125000.0     5170
100000.0     2169
90000.0      1757
80000.0      1436
70000.0      1230
60000.0      1164
50000.0      1027
40000.0       819
30000.0       789
20000.0       784
10000.0       264
5000.0        967
Name: odometer_km, dtype: int64
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


PRICE INFORMATION
(2357,)
99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
Name: price, dtype: int64
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


In [7]:
autos = autos.loc[autos.loc[:,"price"]<3500000,:]

unique_price = autos.loc[:,'price'].unique().shape
price_counts = autos.loc[:,'price'].value_counts().sort_index(ascending = False).head(10)
price_desc = autos.loc[:,'price'].describe()

print('\n')
print ("PRICE INFORMATION")
print(unique_price)
print(price_counts)
print(price_desc)

print(autos.shape)



PRICE INFORMATION
(2351,)
1300000.0    1
1234566.0    1
999999.0     2
999990.0     1
350000.0     1
345000.0     1
299000.0     1
295000.0     1
265000.0     1
259000.0     1
Name: price, dtype: int64
count    4.999100e+04
mean     5.831664e+03
std      1.427337e+04
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.300000e+06
Name: price, dtype: float64
(49991, 20)


## Removing Outliers

After a quick google search it was found that the most expensive car ever sold on ebay was a 1959 Ferrari GT California, which sold for 3,260,100 back in 2010. For this reason, many of the car prices were suspiciously high. It was decided to remove all car prices greater than 3,500,000 to hopefully eliminate any prank ads of cars listed much higher than a reasonable price.

In [8]:
autos.loc[:,"date_crawled"] = autos.loc[:,"date_crawled"].str[:10]
date_crawled_counts = autos.loc[:,"date_crawled"].value_counts(normalize = True, dropna = False).sort_index()

autos.loc[:,"ad_created"] = autos.loc[:,"ad_created"].str[:10]
ad_created_counts = autos.loc[:,"ad_created"].value_counts(normalize = True, dropna = False).sort_index()

autos.loc[:,"last_seen"] = autos.loc[:,"last_seen"].str[:10]
last_seen_counts = autos.loc[:,"last_seen"].value_counts(normalize = True, dropna = False).sort_index()

print ("DATE CRAWLED")
print (date_crawled_counts)
print ('\n')

print ("AD CREATED")
print (ad_created_counts)
print ('\n')

print ("LAST SEEN")
print (last_seen_counts)
print ('\n')

DATE CRAWLED
2016-03-05    0.025385
2016-03-06    0.013943
2016-03-07    0.035966
2016-03-08    0.033266
2016-03-09    0.033206
2016-03-10    0.032126
2016-03-11    0.032486
2016-03-12    0.036767
2016-03-13    0.015563
2016-03-14    0.036627
2016-03-15    0.033986
2016-03-16    0.029505
2016-03-17    0.031526
2016-03-18    0.013062
2016-03-19    0.034906
2016-03-20    0.037827
2016-03-21    0.037507
2016-03-22    0.032926
2016-03-23    0.032386
2016-03-24    0.029105
2016-03-25    0.031746
2016-03-26    0.032486
2016-03-27    0.031046
2016-03-28    0.034846
2016-03-29    0.034166
2016-03-30    0.033626
2016-03-31    0.031906
2016-04-01    0.033806
2016-04-02    0.035406
2016-04-03    0.038687
2016-04-04    0.036507
2016-04-05    0.013102
2016-04-06    0.003181
2016-04-07    0.001420
Name: date_crawled, dtype: float64


AD CREATED
2015-06-11    0.000020
2015-08-10    0.000020
2015-09-09    0.000020
2015-11-10    0.000020
2015-12-05    0.000020
2015-12-30    0.000020
2016-01-03    0.000

## Date Data

The code above is used to extract just the day information to find more about when the data was obtained (crawled) in comparison with when the ad was created and when the ad was last seen.

In [9]:
reg_year_desc = autos.loc[:,"regestration_year"].describe()
reg_year_desc

count    49991.000000
mean      2005.074533
std        105.721987
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: regestration_year, dtype: float64

## Regestration Year

The series above describes certain statistics for the regestration year column of the data. It is noted that the minimum and maximum regestration years are 1000 and 9999 respectively. Considering the first car was invented in the year 1885 (from google) and it is currently the year 2020, it is unlikely that these data points are legitimate.

The first thing that happens with a new car is it gets regestered. Therefore, it is impossible for a car to be first posted online, and then regestered. From the "Last Seen" data above, the most recent ads were last seen in 2016. Any vehicle regestered above this date must be an innacurate data point.

Realistically, the lowest regestration date for a car may be the early 1900's. So we will investigate the number of listings that fall outside the year range 1900-2016.

In [10]:
spooky_reg_autos = autos.loc[(autos.loc[:,'regestration_year']<1900) | (autos.loc[:,'regestration_year']>2016),:]

reg_dates = spooky_reg_autos.loc[:,"regestration_year"]
reg_dates.value_counts(normalize = False).sort_index(ascending = False)


9999       4
9996       1
9000       2
8888       1
6200       1
5911       1
5000       4
4800       1
4500       1
4100       1
2800       1
2019       3
2018     491
2017    1452
1800       2
1500       1
1111       1
1001       1
1000       1
Name: regestration_year, dtype: int64

In [11]:
autos = autos.loc[(autos['regestration_year']>1900) & (autos['regestration_year']<2016),:]

reg_year = autos.loc[:,'regestration_year']
reg_year.value_counts(normalize = True).sort_index(ascending = True)


1910    0.000193
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000043
1937    0.000086
1938    0.000021
1939    0.000021
1941    0.000043
1943    0.000021
1948    0.000021
1950    0.000064
1951    0.000043
1952    0.000021
1953    0.000021
1954    0.000043
1955    0.000043
1956    0.000107
1957    0.000043
1958    0.000086
1959    0.000150
1960    0.000707
1961    0.000128
1962    0.000086
1963    0.000193
1964    0.000257
1965    0.000364
1966    0.000471
1967    0.000578
1968    0.000557
          ...   
1986    0.001627
1987    0.001606
1988    0.003040
1989    0.003875
1990    0.008457
1991    0.007622
1992    0.008372
1993    0.009528
1994    0.014131
1995    0.028091
1996    0.030917
1997    0.043421
1998    0.052521
1999    0.064212
2000    0.071812
2001    0.057852
2002    0.054234
2003    0.058388
2004    0.058602
2005    0.064554
2006    0.057960
2007    0.049331
2008    0.047768
2009    0.044920
2010    0.034193
2011    0.034986
2012    0.028327
2013    0.0172

## Regestration Year - Conclusion

After cleaning the regestration year data to remove unreasonably old cars as well as cars regestered after the ad was seen, there appears to be a vintage cars although the majority of the cars are from the years 1994-2014.

In [12]:
unique_brands = autos.loc[:,'brand'].unique()
unique_brands

array(['peugeot', 'bmw', 'volkswagen', 'smart', 'ford', 'chrysler',
       'seat', 'renault', 'mercedes_benz', 'audi', 'sonstige_autos',
       'opel', 'mazda', 'porsche', 'mini', 'toyota', 'dacia', 'nissan',
       'jeep', 'saab', 'volvo', 'mitsubishi', 'jaguar', 'fiat', 'skoda',
       'subaru', 'kia', 'citroen', 'chevrolet', 'hyundai', 'honda',
       'daewoo', 'suzuki', 'trabant', 'land_rover', 'alfa_romeo', 'lada',
       'rover', 'daihatsu', 'lancia'], dtype=object)

In [25]:
brand_counts = autos.loc[:,'brand'].value_counts(normalize = False)
total_counts = brand_counts.sum()

top_brands = []

for x in unique_brands:
    brand_data = autos.loc[autos.loc[:,'brand'] == x,'brand']
    brand_data_count = brand_data.value_counts().sum()
    
    if ((brand_data_count / total_counts) > 0.05):
        top_brands.append(x)

print(top_brands)

top_brand_prices = {}

for x in top_brands:
    brand_prices = autos.loc[autos.loc[:,'brand'] == x,'price']
    top_brand_prices[x] = brand_prices.mean()
    
top_brand_prices

['bmw', 'volkswagen', 'ford', 'mercedes_benz', 'audi', 'opel']


{'audi': 9131.006156119183,
 'bmw': 8408.143214629452,
 'ford': 4027.1795817958177,
 'mercedes_benz': 8568.267615025561,
 'opel': 2917.0316760224537,
 'volkswagen': 5519.772326406488}

## Top Brands Average Prices

In the code above:

- found total entries
- created empty list to store top brands (brands consisting of greater than 5% of total listings)
- looped through unique brands and added the brand to top_brands if the total listings of that car brand was greater than 5% of the total listings
- looped through top_brands and added it to a dictionary top_brand_prices with the brand as the key and the brands average price as it's value

It is noted that Audi, BMW, Mercedes are significantly higher in price. We will invistigate whether this is a cause of considerably lower mileage or not.

In [28]:
top_brands_avg_mileage = {}

for x in top_brands:
    brand_mileage = autos.loc[autos.loc[:,'brand']==x,'odometer_km']
    top_brands_avg_mileage[x] = brand_mileage.mean()
    
top_brands_avg_mileage

{'audi': 129282.1965033243,
 'bmw': 132314.72569778634,
 'ford': 123825.33825338254,
 'mercedes_benz': 130711.26917092687,
 'opel': 129140.93825180433,
 'volkswagen': 128460.21287379625}

In [36]:
avg_price_series = pd.Series(top_brand_prices)
avg_mileage_series = pd.Series(top_brands_avg_mileage)

price_mileage_data = pd.DataFrame(avg_price_series, columns = ["Mean Price"])
price_mileage_data.loc[:,"Mean Odometer"] = avg_mileage_series

price_mileage_data

,Mean Price,Mean Odometer
audi,9131.006156,129282.196503
bmw,8408.143215,132314.725698
ford,4027.179582,123825.338253
mercedes_benz,8568.267615,130711.269171
opel,2917.031676,129140.938252
volkswagen,5519.772326,128460.212874


# Aggregation Analysis

It appears as though the mean odometer reading (in kilometers) is roughly the same for each of the top listed brands of cars on ebay. This would suggest that many sellers are inflating the value of their car soley based on it's brand. With this being said, with the more luxury brands comes better equipement, more performance etc. Therefore it is inconclusive to say that odometer reading 